# AnacondaでMayaviを使う方法 / How to use Mayavi in Anaconda

1. Anaconda Navigatorで左メニューのEnvironmentsを押し、Create+を押して、Mayaviという名前の新しい環境を作る。環境とは、Pythonライブラリ(パッケージ)を集めたもの。標準はroot環境だが、mayaviは最新環境(3.6)では動かないので、Python3.5を選択する。
2. 最初は最小限(10個)のパッケージしか含まれていない。
2. Mayaviの緑の三角">"を押してOpen Terminalするとターミナルが開く。
3. その中で、OpenCVに必要なものをインストールする。
       conda install -c menpo mayavi=4.5.0
       conda install -c conda-forge pyside=1.2.4
   これにより、新しい環境(Python3.5, Mayavi入り)の準備ができた。今後、また別の開発環境が欲しくなった時も同じ手順を踏む。
5. Anaconda Navigatorに戻り、左メニュー「Home」でアプリ起動画面を表示する。「Applications on」でrootの代わりにMayaviを選ぶと、使えるアプリケーションが更新される。これらのアプリケーションすべてで、Mayaviが利用できる。


1. Press "Environments" in the left menu of the Anaconda Navigator, press "Create[+]" button at the bottom, and make a new environment of Python version 3.5 with a new name "mayavi".  An environment is a collection of packages (libraries).  The standard environment is named "root" and is working on Python version 3.6, but Mayavi library work on Python version 3.5 only, so we prepare another environment.
2. Initially, only 10 packages are included in the Mayavi environment.
3. Press the green triangle at "Mayavi" and select "Open Terminal".
4. In the terminal, type the following command to install the packages required to Mayavi.
        conda install -c menpo mayavi=4.5.0
        conda install -c conda-forge pyside=1.2.4
5. Back to Anaconda Navigator, press "Home" in the left menu to go back the home screen, and select "mayavi" in the "Applications on" selector.  It refreshes the menu item of the Navigator.  You can now use Mayavi in Jupyter and spyder.

### Notes

* https://anaconda.org/menpo/mayavi
* https://github.com/enthought/mayavi/issues/483

Mayaviは三次元グラフを描くためのライブラリです。

非常に美しく描いてくれるので、単にグラフを描くだけでなく、3Dグラフィックス(分子模型など)の表示にも活用できそうです。

動作確認のためのサンプル1。
http://docs.enthought.com/mayavi/mayavi/auto/example_surface_from_irregular_data.html からコピーしました。

In [1]:
# Author: Gael Varoquaux <gael.varoquaux@normalesup.org>
# Copyright (c) 2009, Enthought, Inc.
# License: BSD Style.

import numpy as np

# Create data with x and y random in the [-2, 2] segment, and z a
# Gaussian function of x and y.
np.random.seed(12345)
x = 4 * (np.random.random(500) - 0.5)
y = 4 * (np.random.random(500) - 0.5)


def f(x, y):
    return np.exp(-(x ** 2 + y ** 2))

z = f(x, y)

from mayavi import mlab
mlab.figure(1, fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))

# Visualize the points
pts = mlab.points3d(x, y, z, z, scale_mode='none', scale_factor=0.2)

# Create and visualize the mesh
mesh = mlab.pipeline.delaunay2d(pts)
surf = mlab.pipeline.surface(mesh)

mlab.show()

もう一つのサンプル。Mayaviのサンプルは、[ここ](http://docs.enthought.com/mayavi/mayavi/auto/examples.html)からたくさん入手できます。ざっと見れば、どんなことができるツールかわかってもらえると思います。

In [2]:
import numpy
from mayavi.mlab import *

def test_barchart():
    """ Demo the bar chart plot with a 2D array.
    """
    s = numpy.abs(numpy.random.random((3, 3)))
    return barchart(s)

test_barchart()
mlab.show()

In [1]:
import numpy
from mayavi.mlab import *
from mayavi import mlab


def test_quiver3d(): #quiverとは鏃(やじり)のこと
    x, y, z = numpy.mgrid[-2:3, -2:3, -2:3]
    r = numpy.sqrt(x ** 2 + y ** 2 + z ** 4)
    u = y * numpy.sin(r) / (r + 0.001)
    v = -x * numpy.sin(r) / (r + 0.001)
    w = numpy.zeros_like(z)
    obj = quiver3d(x, y, z, u, v, w, line_width=3, scale_factor=1)
    return obj

test_quiver3d()
mlab.show()

Saving as a vector PS/EPS/PDF/TeX file using GL2PS is either not supported by your version of VTK or you have not configured VTK to work with GL2PS -- read the documentation for the vtkGL2PSExporter class.
Saving as a vector PS/EPS/PDF/TeX file using GL2PS is either not supported by your version of VTK or you have not configured VTK to work with GL2PS -- read the documentation for the vtkGL2PSExporter class.
Saving as a vector PS/EPS/PDF/TeX file using GL2PS is either not supported by your version of VTK or you have not configured VTK to work with GL2PS -- read the documentation for the vtkGL2PSExporter class.


作画自体はPythonですからそれほど速くはありませんが、一旦3Dに変換されたあとは、マウスでぐるぐる回して観察できます。また、ウィンドウから画像(PNG)を生成できます。

## グリッドデータのプロットの例
三次元のグラフを作ります。2次元のリストに単に数値を並べ、それをbarchartに与えると、立体棒グラフで表示してくれます。

In [3]:
from mayavi.mlab import *

data = [[1,2,3,4],[2,3,4,5],[7,6,5,4],[8,7,6,5]]
barchart(data)
mlab.show()

もちろん、numpyのarrayも使えます。以下の例では、van der Waals状態方程式(Sympyでp=の形に変形した式を使います)をプロットします。

In [30]:
import numpy
from mayavi.mlab import *


# p = (T*V**2*k - V*a + a*b)/(V**2*(V - b))
p = numpy.zeros((100,100))  #100x100の空の行列を準備する。

k = 8.314
a = 20000
b = 10
for i in range(100):
    T = float(i+20)
    for j in range(100):
        V = float(j+b+1)
        p[i,j] = (T*V**2*k - V*a + a*b)/(V**2*(V - b))
barchart(p)
mlab.show()

## 散布図の例
`points3d()`を使えば、任意の位置に点(球)を置くことができます。下の例では、x,y,z座標と、その位置での値sをそれぞれnumpy arrayで作成して与えています。points3d関数は、sの値を球の大きさと色合いに変換しています。

In [43]:
from numpy import *
from mayavi.mlab import *

def test_points3d():
    t = linspace(0, 4 * pi, 20)

    x = sin(2 * t)
    y = cos(t)
    z = cos(2 * t)
    s = 2 + sin(t)

    return points3d(x, y, z, s, colormap="copper", scale_factor=.25)

test_points3d()
mlab.show()

## 立体線図の例
点の代わりに線でつなぐこともできます。上のプログラムの、points3dをplot3dに書きかえるだけです。

In [ ]:
from numpy import *
from mayavi.mlab import *

def test_points3d():
    t = linspace(0, 4 * pi, 20)

    x = sin(2 * t)
    y = cos(t)
    z = cos(2 * t)
    s = 2 + sin(t)

    return plot3d(x, y, z, s, tube_radius=0.025, colormap='Spectral')

test_points3d()
mlab.show()

## 立体を描く
棒と球が描けるということは、分子模型を描けるということです。次の例では、ダイヤモンド構造を描きます。

In [ ]:
from numpy import *
from mayavi.mlab import *

SCL = [[x,y,z] for x in (0,4) for y in (0,4) for z in (0,4)]
print(SCL)
FCC = SCL.copy()

# slide toward (2,2,0)
for x,y,z in SCL:
    x += 2
    y += 2
    if x < 4 and y < 4:
        FCC.append([x,y,z])
for x,y,z in SCL:
    y += 2
    z += 2
    if y < 4 and z < 4:
        FCC.append([x,y,z])
for x,y,z in SCL:
    z += 2
    x += 2
    if z < 4 and x < 4:
        FCC.append([x,y,z])
print(FCC)
DIA = FCC.copy()
# slide toward (1,1,1)
for x,y,z in FCC:
    x += 1
    y += 1
    z += 1
    if x < 4 and y < 4 and z < 4:
        DIA.append([x,y,z])
print(DIA)

DIA = numpy.array(DIA) * 10
s = numpy.ones_like(DIA[:,0])
print(s)
points3d(DIA[:,0], DIA[:,1], DIA[:,2], s, scale_factor=10)

#make bonds between adjacent atoms
N = DIA.shape[0]
for i in range(N):
    for j in range(i):
        p1 = DIA[i]
        p2 = DIA[j]
        d = p1 - p2
        if dot(d,d) <= 310:
            # plot a short line segment
            x = [p1[0],p2[0]]
            y = [p1[1],p2[1]]
            z = [p1[2],p2[2]]
            plot3d(x,y,z, tube_radius=3, line_width=10)

mlab.show()

[[0, 0, 0], [0, 0, 4], [0, 4, 0], [0, 4, 4], [4, 0, 0], [4, 0, 4], [4, 4, 0], [4, 4, 4]]
[[0, 0, 0], [0, 0, 4], [0, 4, 0], [0, 4, 4], [4, 0, 0], [4, 0, 4], [4, 4, 0], [4, 4, 4], [2, 2, 0], [2, 2, 4], [0, 2, 2], [4, 2, 2], [2, 0, 2], [2, 4, 2]]
[[0, 0, 0], [0, 0, 4], [0, 4, 0], [0, 4, 4], [4, 0, 0], [4, 0, 4], [4, 4, 0], [4, 4, 4], [2, 2, 0], [2, 2, 4], [0, 2, 2], [4, 2, 2], [2, 0, 2], [2, 4, 2], [1, 1, 1], [3, 3, 1], [1, 3, 3], [3, 1, 3]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


単に球と棒を描く手段としては、ちょっと面倒くさい気もしますね。

## 三次元の等高面を描く
二次元の定義域に対する関数は、三次元のグラフで表現できます。では、三次元に分布する関数はどうやって表現するか。等高面を描けば良いのです。


## アニメーション
アニメーションはいまのところうまく動かせていません。

In [1]:
from mayavi import mlab
@mlab.animate
def anim():
    f = mlab.gcf()
    while 1:
        f.scene.camera.azimuth(10)
        f.scene.render()
        yield

a = anim() # Starts the animation.

In [ ]:
import numpy as np
from mayavi import mlab

# Produce some nice data.
n_mer, n_long = 6, 11
pi = np.pi
dphi = pi/1000.0
phi = np.arange(0.0, 2*pi + 0.5*dphi, dphi, 'd')
mu = phi*n_mer
x = np.cos(mu)*(1+np.cos(n_long*mu/n_mer)*0.5)
y = np.sin(mu)*(1+np.cos(n_long*mu/n_mer)*0.5)
z = np.sin(n_long*mu/n_mer)*0.5

# View it.
l = mlab.plot3d(x, y, z, np.sin(mu), tube_radius=0.025, colormap='Spectral')
# Now animate the data.
ms = l.mlab_source
for i in range(10):
    x = np.cos(mu)*(1+np.cos(n_long*mu/n_mer +
                                      np.pi*(i+1)/5.)*0.5)
    scalars = np.sin(mu + np.pi*(i+1)/5)
    ms.set(x=x, scalars=scalars)
    mlab.show()


次のプログラムは松本が作った実例です。

In [11]:
import numpy as np
from mayavi.mlab import *
import colorsys



def tangents(x,y,z):
    v = np.array([x,y,z]).transpose()
    v0 = v[0:v.shape[0]-2]
    v1 = v[2:]
    dv = v1 - v0
    tangent = np.zeros_like(v)
    for i in range(dv.shape[0]):
        tangent[i+1] = dv[i] / np.linalg.norm(dv[i])
    tangent[0] = tangent[1]
    tangent[-1] = tangent[-2]
    return tangent

def normals(x,y,z):
    v = np.array([x,y,z]).transpose()
    v0 = v[0:v.shape[0]-1]
    v1 = v[1:]
    dv = v1 - v0
    dv0 = dv[0:dv.shape[0]-1]
    dv1 = dv[1:]
    n0 = np.cross(dv0,dv1)
    n1 = np.zeros_like(v)
    for i in range(n0.shape[0]):
        n1[i+1] = n0[i] / np.linalg.norm(n0[i])
    n1[0] = n1[1]
    n1[-1] = n1[-2]
    return n1


def ribbon(x,y,z,width,twist, arrow_rows=0, arrow_expansion=1.0):
    norm = normals(x,y,z)
    tang = tangents(x,y,z)
    v3   = np.cross(norm,tang)
    #plot3d(x,y,z)
    v = np.array([x,y,z]).transpose()
    sine   = np.sin(twist)
    cosine = np.cos(twist)
    direction = np.zeros_like(v)
    for i in range(v.shape[0]):
        direction[i] = width[i]*(cosine[i]*norm[i,:] + sine[i]*v3[i,:])
    ps = []
    qs = []
    for i in range(v.shape[0]):
        j = v.shape[0] - i - 1
        if j <= arrow_rows and arrow_rows != 0:
            arrow = arrow_expansion*j/arrow_rows
        else:
            arrow = 1
        p = v[i]-direction[i]*arrow/2
        q = v[i]+direction[i]*arrow/2
        ps.append(p)
        qs.append(q)
    qs.reverse()
    pq = ps + qs
    
    triplets = []
    for p in range(v.shape[0]-1):
        q = v.shape[0]*2 - p - 1
        triplets.append([p,p+1,q])
        triplets.append([p+1,q-1,q])
    xx,yy,zz = np.array(pq).transpose()
    return xx,yy,zz,triplets
        
        #xx,yy,zz = np.array([p,q]).transpose()
        #plot3d(xx,yy,zz)


def spiral_tower(r,h,n):
    """Generates a pretty set of lines."""
    t = np.arange(0.0, 6.0, 0.001)
    pi = np.pi
    mu = t*pi*1
    twist = t*0-0.3 #pi*0.2
    width = t*0+0.1
    z = t - 3
    thick = np.exp(-t/3)
    thick[0] = 0
    for i in range(n):
        x = np.cos(mu+2*pi*i/n) * (r/(t+0.3) - 0.12)
        y = np.sin(mu+2*pi*i/n) * (r/(t+0.3) - 0.12)
        tx,ty,tz,ta = ribbon(x,y,z,thick,twist,arrow_rows=600,arrow_expansion=2.0)
        plot3d(tx,ty,tz)
        h = float(i)/n + 0.3
        if h > 1:
            h -= 1
        l = triangular_mesh(tx, ty, tz, ta, color=colorsys.hsv_to_rgb(h,1,1))
    return l

clf()
spiral_tower(1,1,6)
#points3d(0,0,60,1,color=(1,1,1))
#test_points3d()
mlab.show()

In [ ]:
l